# MC of a Compton interaction: part II

## Introduction

In the previous part, you implemented a method to sample the distance travelled before a compton interaction and the $\theta$ and $\phi$ scatter angle, after the interaction has taken place.

Now we can put everything together and run our small MC experiment. 


## 1. Recap of functions you wrote yourself in part 1

#### packages

In [ ]:
import numpy as np
import sympy
import matplotlib.pyplot as plt
plt.style.use('seaborn-colorblind')

#### 1.1 Distance travelled

In [ ]:
# DO NOT CHANGE

def get_compton_crosssection(energy_keV, atomic_number, density):
    """reads the nist file, interpolates to energy = energy_keV and return the associated compton cross section
    energy_keV = energy [keV]
    atomic number = Z value
    density = density in [g/cm³]
    returns: cross section [1/cm]
    """ 
    if energy_keV< 1:
        print('Energy should be larger or equal to 1 keV')
        return 0
    # read in NIST DATA
    input = np.loadtxt('CrossSection_Compton_Z%i.txt' %atomic_number, skiprows =3)
    # set energy in MeV
    energy_MeV = energy_keV/1000
    # interpolate
    cross_section = np.interp(energy_MeV, input[:,0], input[:,1])*density
    return cross_section

def distance_travelled_before_interaction(mu):
    """ 
    input: cross section for compton scatter [1/cm]
    returns: sampled distance travelled before interaction [cm]
    """
    xi = np.random.random(1)  # one random value between [0,1]
    s = -1/mu * np.log(1-xi)
    return s

#### 1.2 The scatter angles

In [ ]:
# DO NOT CHANGE
''' Kappa
'''
mec2 = 5.11e5 # rest energy electron eV
kappa = lambda E: E * 1000/mec2  # a function that calculates kappa from an incoming energy E in keV

def get_pdf(energy_keV):
    """ the pdf in a function
    """
    tau_s = sympy.symbols('tau', real=True, positive=True) # ratio of the scattered and incoming energy Ec/E
    k_s = sympy.symbols('kappa', real=True, positive=True) # ratio of the incoming energy and the electron rest energ
    pdf_symp = 1/tau_s**2 + (k_s**2-2*k_s-2)/tau_s + (2*k_s+1) + k_s**2*tau_s
    pdf_symp_kfix = pdf_symp.subs(k_s, kappa(energy_keV))  # subsitute 'k_s' by a value 
    min_tau = 1/(1+2*kappa(energy_keV))
    max_tau = 1
    max_value = pdf_symp_kfix.subs(tau_s, min_tau) 
    pdf_symp_kfix = pdf_symp_kfix/max_value
    pdf = sympy.lambdify([tau_s], pdf_symp_kfix)
    return pdf, min_tau, max_tau


def rejection_sampling(min_x, max_x, fl_x, print_val = False):
    """ 
    input
    return: [boolean, sample]
    """
    [xi1, xi2] = np.random.random(2)
    x = min_x + (max_x-min_x)*xi1
    if print_val:
        print('x=',x, " and xi2 =", xi2)
    if xi2<=fl_x(x):
        return True, x
    else:
        return False, x
    
def get_phi():
    xi =np.random.random(1)
    phi = np.pi*2*xi
    return phi

def get_theta(tau, k):
    cos_theta = 1-(1-tau)/(k*tau)
    return np.arccos(cos_theta)

#### 1.3 From angle to a count on the detector

In [ ]:
def get_direction(phi, theta):
    de = np.zeros(3)
    de[0] = np.sin(theta)*np.cos(phi)
    de[1] = np.sin(theta) * np.sin(phi)
    de[2] = np.cos(theta)
    return de

def get_detector_position_co(co_i, d, z_detector=10, xs_detector=100, ys_detector=100, print_val=False):
    distance_traveled = (z_detector - co_i[2])/d[2]
    if not distance_traveled >0:
        return None
    else:
        co = distance_traveled*d+co_i
        if abs(co[0]) < xs_detector/2 and abs(co[1]) < ys_detector/2:
            return co
        else:
            if print_val:
                print('out of bounce detector: (',co[0], ',', co[1],')')
            return None
        
class Image:
    def __init__(self, xs_detector, ys_detector, pixelsize,print_val=False):
        self.pixelsize = pixelsize
        self.n_pix_x = int(xs_detector//pixelsize)
        self.n_pix_y = int(ys_detector // pixelsize)
        self.image = np.zeros([self.n_pix_x, self.n_pix_y])
        self.xs_detector = pixelsize * self.n_pix_x
        self.ys_detector = pixelsize * self.n_pix_y
        self.shift_co = np.array([self.xs_detector/2, self.ys_detector/2, 0])
        self.print = print_val
        
    def add_count(self, co):
        co = co + self.shift_co
        pix_idx = int(co[0]//self.pixelsize)
        pix_idy = int(co[1] // self.pixelsize)
        self.image[pix_idx,pix_idy] += 1
        if self.print:
            print('co: ', co[0], ', ',co[1], "added at [",pix_idx, ',',pix_idy,']')
        return

## 2. MC simulation

In [ ]:
def mc_compton(n, incoming_energy_keV, mu, thickness_slab, z_d, xs_detector, ys_detector, pixelsize):
    """ Simulate N Compton events 
    
    INPUT
    * n = number of iterations
    * incoming_energy_keV = the energy of the incoming photon in keV
    * mu = the cross section for compton in 1/cm
    * thickness_slab = the thickness of the material in cm
    * z_d = the position of the detector in cm
    * xs_detector = the length of the detector in cm in the x direction
    * ys_detector = the length of the detector in cm in the y direction
    * pixelsize = the x and y length of a pixel in cm
    
    return: /
    """
    
    # We will keep track of how many photons are 
    # - rejected when doing the angle sampling, 
    # - considered primaries (and therefore ignored) 
    # - or are scattered out of bounce
    n_out_of_bounce = 0
    n_rejected = 0
    n_primary = 0 
    
    # get the pdf distribution for the energy of the incoming photon, calculate kappa for this energy and initialise the image
    pdf, min_tau, max_tau = get_pdf(incoming_energy_keV)
    kappa_v = kappa(incoming_energy_keV)
    im = Image(xs_detector=xs_detector, ys_detector=ys_detector,pixelsize=pixelsize)
    
    # Run the events 
    for i in range(n):
        # compton interaction in the slab
        s = distance_travelled_before_interaction(mu)
        if s>thickness_slab:
            n_primary +=1
            continue
        co_i = np.array([0,0,s[0]])
        
        # compton interaction in the slab
        res, tau_out = rejection_sampling(min_tau, max_tau, pdf)
        if res:
            # Get the angles
            phi_out = get_phi()
            theta_out = get_theta(tau_out, kappa_v)
            # Get the direction
            d = get_direction(phi_out, theta_out) 
            # Get the position of the scattered photon on the detector
            co_e = get_detector_position_co(co_i, d, z_detector=z_d, xs_detector=im.xs_detector, ys_detector=im.ys_detector)
            if co_e is not None:
                im.add_count(co_e)
            else:
                n_out_of_bounce +=1
        else:
            n_rejected +=1 


    # Show the image
    plt.imshow(im.image)
    plt.show()
    
    # Calculate the rejection rates
    if n==n_primary:
        rejectionrate = 0
    else:
        rejectionrate = np.round(n_rejected/(n-n_primary)*100)
    if n==n_primary+n_rejected: 
        outofbouncerate = 0
    else:
        outofbouncerate = np.round(n_out_of_bounce/(n-n_primary-n_rejected)*100)
    primaryrate = np.round(n_primary/n*100)
    print('Angle rejection Rate = ', rejectionrate,'%, out of bounce rate=', outofbouncerate ,'% Primary rate=', primaryrate , ' %')

**excercise:** Explain
1. The effect of energy on the rejection rate when sampling $\theta $. 
2. The effect of slab thickness on the primary rate.
3. The effect of detector distance on the out of bounce rate and primary rate

In [ ]:
incoming_energy_keV = 1000
mu = get_compton_crosssection(incoming_energy_keV, atomic_number=20, density=1.63)

mc_compton(n=20000,
           incoming_energy_keV= incoming_energy_keV, 
           mu=mu, 
           thickness_slab = 2, 
           z_d=23, 
           xs_detector=100, 
           ys_detector=100, 
           pixelsize=1)